## Levatamos Fermionic

In [ ]:
import numpy as np
import openfermion as of
from tqdm import tqdm
from itertools import combinations
from openfermion.utils import commutator, count_qubits, hermitian_conjugated
import functools
import concurrent.futures
from numba import njit
import tensorflow as tf
import scipy
import sparse
import itertools
import linecache

# Generación de base
class fixed_basis:
    @staticmethod
    def int_to_bin(k, d):
        return np.base_repr(k, 2).zfill(d)

    @staticmethod
    def bin_to_op(b):
        tups = [(i, 1) for i, k in list(enumerate(list(b))) if k == '1']
        return of.FermionOperator(tups)

    def idx_to_repr(self, idx):
        return self.canonicals[idx]

    def opr_to_idx(self, opr):
        for i in range(self.size): # Evitar esto ordenando opr
            if self.base[i] == opr:
                return i

    # Calcula el valor medio a partir del indice del vector y el operador
    def idx_mean_val(self, idx: int, op: of.FermionOperator):
        vec = self.idx_to_repr(idx)
        return np.real(np.transpose(vec) @ of.get_sparse_operator(op, n_qubits=self.d) @ vec)

    # Calcula el valor medio a partir de un estado y el operador
    def mean_val(self, vec, op):
        idx = self.opr_to_idx(vec)
        return self.idx_mean_val(idx, op)

    # Calcula la contracción de un operador sobre dos estados dados
    def idx_contraction(self, idx_1, idx_2, op):
        rep = lambda x: self.idx_to_repr(x)
        return np.real(np.transpose(rep(idx_1)) @ of.get_sparse_operator(op, n_qubits=self.d) @ rep(idx_2))

    def create_basis(self, d, num = None, pairs = False):
        basis = []
        num_ele = []
        for k in range(0,2**d):
            b = self.int_to_bin(k, d)
            if num != None:
                if b.count('1') == num:
                    if pairs:
                        if np.all(b[::2] == b[1::2]):
                            oper = self.bin_to_op(b)
                            basis.append(oper)
                            num_ele.append(k)
                    else:
                        oper = self.bin_to_op(b)
                        basis.append(oper)
                        num_ele.append(k)
            else:
                oper = self.bin_to_op(b)
                basis.append(oper)
        return basis, num_ele

    def __init__(self, d, num = None, pairs = False, basis = None, num_ele = None):
        self.d = d
        self.num = num
        self.m = num
        # Si nos da la base, la levantamos (asumimos GC). Si no, la creamos
        if basis is None:
            self.base, self.num_ele = self.create_basis(d, num, pairs)
        else:
            self.base, self.num_ele = basis, num_ele
        self.size = len(self.base)
        self.canonicals = np.eye(self.size)
        self.pairs = pairs

    @staticmethod
    def cdc(i, j):
        return of.FermionOperator(((i,1),(j,0)))

    @staticmethod
    def cc(i, j):
        return of.FermionOperator(((i,0),(j,0)))

    # Del indice, cuenta el número de partículas
    def num_idx(self, idx):
        b = self.int_to_bin(idx, basis.d)
        return b.count('1')

    # Calculo de rho1 (via directa, lento, y solo definido en la base por ahora)
    def rho_1(self, op):
        # Necesitamos un índice, es?
        if type(op) != int:
            op = self.opr_to_idx(op)
        mat = np.zeros((self.d, self.d))
        for i in range(self.d):
            for j in range(self.d):
                cdc = self.cdc(j, i)
                mat[i,j] = self.idx_mean_val(op, cdc)
        return mat

# Calculo de generadores de rho1
def rho_1_gen(basis):
    # Vamos a crear un hipersparse de TF, almacenamos los valores acá
    indices = []
    values = []
    shape = (basis.d, basis.d, basis.size, basis.size)
    d = basis.d
    for i in tqdm(range(0, d)):
        for j in range(0, d):
            # Generamos el operador
            op = basis.cdc(j, i)
            #print(op)
            if basis.num == None:
                mat = np.real(of.get_sparse_operator(op, n_qubits=d))
            else:
                mat = np.real(of.get_sparse_operator(op, n_qubits=d))[np.ix_(basis.num_ele, basis.num_ele)]
            # Extraemos la información
            n_r, n_c = mat.nonzero()
            data = mat.data
            for r, c, v in zip(n_r, n_c, data):
                indices.append([i, j, r, c])
                values.append(v)
    indices_t = np.array(indices).T
    s_t = sparse.COO(indices_t, values, shape=shape)
    return s_t

# Calculo de rho1 (via generadores) de un vector en la base canonica
def rho_1(vect, rho_1_arrays):
    if len(vect.shape) == 1: # vectores
        return sparse.einsum('k,ijkl,l->ij', vect, rho_1_arrays, vect)
    elif len(vect.shape) == 2: # mat densidad
        return sparse.einsum('ijkl,kl->ij', rho_1_arrays, vect)
    else: # mat densidad batcheadas
        return sparse.einsum('bkl,ijkl->bij', vect, rho_1_arrays)

# Calculo de indices de rho2kkbar
def get_kkbar_indices(t_basis):
    indices = []
    for i, ind in enumerate(t_basis.num_ele):
        v = t_basis.int_to_bin(ind, t_basis.d)
        if np.all(v[::2] == v[1::2]):
            indices.append(i)
    return indices

# Calculo de generadores de rho2
def rho_2_gen(basis, t_basis, idx_list = []):
    # Vamos a crear un hipersparse de TF, almacenamos los valores acá
    d = basis.d
    indices = []
    values = []
    if len(idx_list) == basis.m:
        idx_list = idx_list
    elif len(idx_list) == basis.m**4:
        idx_list = np.unique(idx_list[:,0])
    else:
        idx_list = range(t_basis.size)
    shape = (len(idx_list), len(idx_list), basis.size, basis.size)
    for i, ii in tqdm(enumerate(idx_list), total=len(idx_list)):
        for j, jj in enumerate(idx_list):
            # Generamos el operador
            op = t_basis.base[jj]*of.utils.hermitian_conjugated(t_basis.base[ii])
            if basis.num == None:
                mat = np.real(of.get_sparse_operator(op, n_qubits=d))
            else:
                mat = np.real(of.get_sparse_operator(op, n_qubits=d))[np.ix_(basis.num_ele, basis.num_ele)]
            # Extraemos la información
            n_r, n_c = mat.nonzero()
            data = mat.data
            for r, c, v in zip(n_r, n_c, data):
                indices.append([i, j, r, c])
                values.append(v)

    indices_t = np.array(indices).T
    s_t = sparse.COO(indices_t, values, shape=shape)
    return s_t

def rho_m_gen(basis, m):
    indices = []
    values = []
    m_basis = fixed_basis(basis.d, num=m, pairs=basis.pairs)
    shape = (m_basis.size, m_basis.size, basis.size, basis.size)
    
    it_set = np.arange(m_basis.size) #[::-1]
    for i, ii in tqdm(enumerate(it_set), total=m_basis.size):
        for j, jj in enumerate(it_set):
            # Generamos el operador
            op = m_basis.base[jj]*of.utils.hermitian_conjugated(m_basis.base[ii])
            mat = np.real(of.get_sparse_operator(op, n_qubits=basis.d))[np.ix_(basis.num_ele, basis.num_ele)]
            # Extraemos la información
            n_r, n_c = mat.nonzero()
            data = mat.data
            for r, c, v in zip(n_r, n_c, data):
                indices.append([i, j, r, c])
                values.append(v)

    indices_t = np.array(indices).T
    s_t = sparse.COO(indices_t, values, shape=shape)
    return s_t

def rho_m(vect, rho_m_arrays):
    return sparse.einsum('k,ijkl,l->ij', vect, rho_m_arrays, vect)

# Calculo de rho2 (via generadores) de un estado en la base canonica
def rho_2(vect, rho_2_arrays):
    if len(vect.shape) == 1: # vectores SOLO RHO2 COMPLETA
        return sparse.einsum('k,ijkl,l->ij', vect, rho_2_arrays, vect)
    elif len(vect.shape) == 2: # mat densidad SOLO RHO2 COMPLETA
        return sparse.einsum('ijkl,kl->ij', rho_2_arrays, vect)
    else: # mat densidad batcheadas
        return sparse.einsum('bkl,ijkl->bij', vect, rho_2_arrays)

# Calculo de generadores de K (usado para quasiparticles) WIP SPARSE
def k_gen(basis):
    mat = np.zeros((basis.d, basis.d, basis.size, basis.size))
    d = basis.d
    for i in tqdm(range(0, d), total=d):
        for j in range(0, d):
            op = basis.cc(j, i)
            if basis.num == None:
                mat[i,j,::] = np.real(of.get_sparse_operator(op, n_qubits=d)).todense()
            else:
                mat[i,j,::] = np.real(of.get_sparse_operator(op, n_qubits=d)).todense()[np.ix_(basis.num_ele, basis.num_ele)]
    return mat

def k_vect(vect, k_gen):
    return np.einsum('k,ijkl,l->ij', vect, k_gen, vect)

# Calculo la matrix rho de cuasipartículas  WIP SPARSE
def rho_qsp(vect, rho_1_arrays, k_arrays, rho1 = None):
    if type(rho1) == None:
        rho1 = rho_1(vect, rho_1_arrays)
    k = k_vect(vect, k_arrays)

    mat = np.block([[rho1, k], [-np.conjugate(k), np.eye(rho_1_arrays.shape[0])-np.conjugate(rho1)]])
    return mat

# Devuelve los indices que tienen a level ocupado
def level_proy(d, level):
    ids = []
    for k in range(0,2**d):
        b = fixed_basis.int_to_bin(k, d)
        if b[level] == '1':
            ids.append(k)
    arr = np.zeros(2**d)
    arr[np.array(ids)] = 1
    return arr, ids

def parity_levels(d):
    rng = range(2**d)
    binary_repr = np.vectorize(np.binary_repr)(rng)
    ones_c = np.char.count(binary_repr, '1')
    return np.array(rng)[ones_c % 2 == 1] # seleccionamos estados impares

# Devuelve el vector postmedido
def measure(basis, vect, level = 1):
    l_arr, l_ids = level_proy(basis.d, level)
    proy_v = vect * l_arr
    comp_arr = np.logical_not(l_arr).astype(int)
    comp_v = vect * comp_arr
    norm = lambda v: v / np.linalg.norm(v)
    return norm(proy_v), norm(comp_v)

def entropy(rho, m):
    S_fun = lambda rho: -1*np.trace(rho @ scipy.linalg.logm(rho)) / np.log(2)
    ent = S_fun(rho) / (np.log2(scipy.special.binom(basis.d, m)))
    return ent

# Levanta bases de QChem
def build_csv_basis(csvf, d):
    # Construimos la base
    ops = []
    with open(csvf, 'r') as basis:
        num_ele = []
        # Contar los niveles
        m_level = 0
        # Creamos operadores
        for l in basis.read().splitlines()[4:]:
            natop = [int(x) for x in l.split(' ')[1:]] # Operador en forma de lista
            op = of.FermionOperator(([(i, 1) for i in natop]))
            ops.append(op)
            # Contamos niveles
            m_level = max(m_level, *natop)
            # Determinamos el índice
            natop_to_int = lambda x: np.sum([2**(d-1-i) for i in x])
            num_ele.append(natop_to_int(natop))
 
        # Determinamos m y d
        num = len(natop)
        assert d == m_level+1

    return fixed_basis(d, num = num, pairs = False, basis = ops, num_ele=num_ele)
        

2024-12-19 14:44:54.229839: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-19 14:44:54.229893: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-19 14:44:54.297679: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-19 14:44:54.451476: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-19 14:44:55.730194: W tensorflow/compiler/tf2

In [ ]:
basis = build_csv_basis('hf_basis', 12)

rho_1_arrays = rho_m_gen(basis, 1)
rho_2_arrays = rho_m_gen(basis, 2)
# Indices
t_basis = fixed_basis(basis.d, num=2)
k_indices = get_kkbar_indices(t_basis)
print(k_indices)
# Arrays reducidos
rho_2_kkbar_arrays = rho_2_gen(basis, t_basis, idx_list = k_indices)


100%|██████████| 66/66 [02:48<00:00,  2.55s/it]


[0, 5, 14, 27, 44, 65]


100%|██████████| 66/66 [02:54<00:00,  2.65s/it]


### Determinación de estado

In [8]:
from scipy.cluster.hierarchy import fcluster, linkage
from scipy.cluster.vq import kmeans, vq

# Input
rho_1_obj  = np.sort(np.concatenate([np.repeat(1/2, 2*2), np.repeat(1, 4*2)]))
rho_2_obj = np.sort(np.concatenate([np.repeat(1/2, 8*2+16), np.repeat(1, 6*2+17), np.repeat(0, 5)]))
vect = [7.211630245189536e-01,4.206455224775346e-02,-2.285388031865260e-12,1.699211387138645e-12,-4.129479393831875e-03,-3.823465330043704e-07,4.206455225178431e-02,-6.901551297130625e-01,2.105502564059137e-12,-1.565639815440850e-12,-4.122956845587538e-03,-3.755879654736101e-05,-7.228479653547532e-12,6.636332640706462e-12,-9.140952135390417e-04,-4.046086775026300e-16,8.239170289440526e-14,2.416202424789693e-16,-8.029397050081800e-13,7.469929098902930e-13,-1.075293384255381e-14,-9.140952135362526e-04,4.469835410063215e-15,5.163385528935227e-17,-4.129479393815261e-03,-4.122956845567992e-03,3.604262375317366e-14,-2.262779577530237e-14,-1.075992639441983e-03,-2.588630943109245e-05,-3.823465329666192e-07,-3.755879654741322e-05,5.671459804258443e-17,-2.094441474950918e-16,-2.588630943109360e-05,-8.771828180833955e-06]

# Aux functions
r_eig = lambda x: np.sort(np.real(np.linalg.eigvals(x.todense())))
k = 2 # Número de clusters

def sparsity_error(A, k=2):
    # L1
    sparsity_term = np.sum(np.abs(A))

    # Clustering 
    non_zero_coeffs = A[np.abs(A) > 1e-5]  # Sacamos los ceros
    if len(non_zero_coeffs) > 0:
        # Calculamos los clusters (k) y la distancia en ellos
        centroids, _ = kmeans(non_zero_coeffs, k)
        cluster_labels, distances = vq(non_zero_coeffs, centroids)
        cluster_error = np.sum(distances**2) 
    else:
        cluster_error = 0

    loss = 1 * sparsity_term + 2 * cluster_error
    return loss

# Loss definition
def loss_fun(vect):
    # Rho 2 loss
    rho = rho_m(vect, rho_2_arrays)
    eigv = r_eig(rho)
    lrho2 = np.linalg.norm(rho_2_obj-eigv)
    # Rho 1 loss
    rho = rho_m(vect, rho_1_arrays)
    eigv = r_eig(rho)
    lrho1 = np.linalg.norm(rho_1_obj-eigv)
    # L1 + clustering loss
    l1 = sparsity_error(vect, k) 

    print(lrho2, lrho1, l1)
    return lrho2 + lrho1 + 1/2*l1

opt = scipy.optimize.minimize(loss_fun, vect, method = 'Nelder-Mead')
print(opt)
print(opt.x)
print(rho_2_obj)
print(sparsity_error(opt.x, k=k))
r_eig(rho_m(opt.x, rho_2_arrays))

0.12403163550558245 0.0438437411432223 2.418936556508627
0.42519184884466754 0.18716394934892855 2.454994707734575
0.12433275334722096 0.04395185870492564 2.421777798687072
0.12403163550558167 0.04384374114322247 2.4189365565087417
0.1240316355055817 0.04384374114322247 2.4189365565087124
0.12403687064772596 0.04384558488074779 2.419109616667279
0.1240316355058968 0.043843741143317475 2.418936575625954
0.12433275334945935 0.04395185870571809 2.421777798687379
0.29732494820584593 0.14655318547189722 2.508670465578371
0.12403163550558127 0.043843741143222745 2.4189365565087324
0.12403163550558145 0.04384374114322247 2.4189365565087053
0.12402697630055642 0.043842100809899635 2.4191093378139756
0.12403163509162499 0.04384374099838502 2.4189380990892113
0.12403163550558224 0.043843741143222634 2.4189365565089886
0.12403163550558154 0.0438437411432228 2.4189365565089593
0.12403232470367997 0.0438437411473237 2.418974263359842
0.12403163550558201 0.0438437411432223 2.418936556508627
0.124031

array([2.77494681e-06, 2.77494681e-06, 2.77494681e-06, 2.77494681e-06,
       2.77494681e-06, 4.99997515e-01, 4.99997515e-01, 4.99997515e-01,
       4.99997515e-01, 4.99997685e-01, 4.99997685e-01, 4.99997685e-01,
       4.99997685e-01, 4.99997821e-01, 4.99997821e-01, 4.99997821e-01,
       4.99997821e-01, 4.99998598e-01, 4.99998598e-01, 4.99998598e-01,
       4.99998598e-01, 5.00013273e-01, 5.00013273e-01, 5.00013273e-01,
       5.00013273e-01, 5.00013443e-01, 5.00013443e-01, 5.00013443e-01,
       5.00013443e-01, 5.00013579e-01, 5.00013579e-01, 5.00013579e-01,
       5.00013579e-01, 5.00014357e-01, 5.00014357e-01, 5.00014357e-01,
       5.00014357e-01, 9.98344802e-01, 1.00000817e+00, 1.00000818e+00,
       1.00000818e+00, 1.00000818e+00, 1.00000818e+00, 1.00000832e+00,
       1.00000832e+00, 1.00000832e+00, 1.00000832e+00, 1.00000849e+00,
       1.00000849e+00, 1.00000849e+00, 1.00000849e+00, 1.00000852e+00,
       1.00000910e+00, 1.00000910e+00, 1.00000910e+00, 1.00000910e+00,
      

In [349]:
est = 0
for i, ii in enumerate(opt.x):
    if np.abs(ii) > 0.0001:
        print(i, ii, basis.base[i])
        if np.abs(ii) > 0.2:
            est += ii * basis.canonicals[i]


#(1/np.sqrt(3), 1/np.sqrt(12))
print(est)
print(r_eig(rho_m(est, rho_2_arrays)))

#est = 1/np.sqrt(3) * (basis.canonicals[0] - basis.canonicals[35]) + 1/np.sqrt(12) * (basis.canonicals[28]-basis.canonicals[7]+basis.canonicals[15]-basis.canonicals[20])
#print(r_eig(rho_m(est, rho_2_arrays)))


0 0.7059358124833219 1.0 [0^ 1^ 2^ 3^ 4^ 6^ 7^ 8^ 9^ 10^]
1 0.040235316271977156 1.0 [0^ 1^ 2^ 3^ 4^ 6^ 7^ 8^ 9^ 11^]
4 -0.003845897576731483 1.0 [0^ 1^ 2^ 3^ 4^ 6^ 8^ 9^ 10^ 11^]
6 0.04023946945913235 1.0 [0^ 1^ 2^ 3^ 5^ 6^ 7^ 8^ 9^ 10^]
7 -0.7059481115042943 1.0 [0^ 1^ 2^ 3^ 5^ 6^ 7^ 8^ 9^ 11^]
10 -0.0039298315011113485 1.0 [0^ 1^ 2^ 3^ 5^ 6^ 8^ 9^ 10^ 11^]
14 -0.000955887583496742 1.0 [0^ 1^ 2^ 4^ 5^ 6^ 7^ 8^ 10^ 11^]
21 -0.0008817536304567353 1.0 [0^ 1^ 3^ 4^ 5^ 6^ 7^ 9^ 10^ 11^]
24 -0.003919982468903605 1.0 [0^ 2^ 3^ 4^ 5^ 6^ 7^ 8^ 9^ 10^]
25 -0.003949961264520739 1.0 [0^ 2^ 3^ 4^ 5^ 6^ 7^ 8^ 9^ 11^]
28 -0.0010386081326971357 1.0 [0^ 2^ 3^ 4^ 5^ 6^ 8^ 9^ 10^ 11^]
[ 0.70593581  0.          0.          0.          0.          0.
  0.         -0.70594811  0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0

In [259]:
r_eig(rho_m(vect, rho_2_arrays))

array([6.39649654e-18, 2.27759896e-17, 2.27759896e-17, 2.97340303e-17,
       1.05847838e-08, 1.05847839e-08, 8.12469946e-02, 8.12469946e-02,
       8.12469946e-02, 8.12469946e-02, 8.12469946e-02, 8.12469946e-02,
       8.26670825e-02, 8.26670825e-02, 8.26670825e-02, 8.26670825e-02,
       8.26670825e-02, 8.26670825e-02, 3.36085923e-01, 3.36085923e-01,
       3.36085923e-01, 3.36085923e-01, 3.36085923e-01, 3.36085923e-01,
       7.48001237e-01, 7.48001237e-01, 7.51998753e-01, 7.51998753e-01])

#### Testing

In [256]:
vect = [-4.646928018967011e-01,2.770294629368069e-13,6.042253100720720e-13,-6.024516357133285e-13,-2.567187786718633e-13,1.437594192304828e-01,-2.763730452330614e-13,4.646928018967681e-01,6.783135520862164e-13,6.755585098225312e-13,-1.437594192304206e-01,2.565069368880155e-13,-6.036534951757788e-13,-6.777043000828106e-13,-6.255919748617283e-14,-2.875188384604817e-01,-6.292320178844996e-13,-5.606330583371935e-13,6.014298958703158e-13,-6.759194003610720e-13,-2.875188384604817e-01,-6.207393804663961e-14,-6.269833844031960e-13,5.590088393571927e-13,2.557458444682729e-13,-1.437594192304207e-01,6.287588334290853e-13,6.270036860946324e-13,4.000749631766127e-01,-2.383049928519640e-13,1.437594192304827e-01,-2.559215089660516e-13,5.600763473792744e-13,-5.580405389843103e-13,2.382296361695720e-13,-4.000749631765536e-01]
vect = np.array(vect)
vect

array([-4.64692802e-01,  2.77029463e-13,  6.04225310e-13, -6.02451636e-13,
       -2.56718779e-13,  1.43759419e-01, -2.76373045e-13,  4.64692802e-01,
        6.78313552e-13,  6.75558510e-13, -1.43759419e-01,  2.56506937e-13,
       -6.03653495e-13, -6.77704300e-13, -6.25591975e-14, -2.87518838e-01,
       -6.29232018e-13, -5.60633058e-13,  6.01429896e-13, -6.75919400e-13,
       -2.87518838e-01, -6.20739380e-14, -6.26983384e-13,  5.59008839e-13,
        2.55745844e-13, -1.43759419e-01,  6.28758833e-13,  6.27003686e-13,
        4.00074963e-01, -2.38304993e-13,  1.43759419e-01, -2.55921509e-13,
        5.60076347e-13, -5.58040539e-13,  2.38229636e-13, -4.00074963e-01])

In [167]:
np.sort(np.linalg.eigvals(rho_m(vect, rho_2_arrays).todense()))

array([6.39649654e-18+0.00000000e+00j, 2.27759896e-17-1.60872919e-17j,
       2.27759896e-17+1.60872919e-17j, 2.97340303e-17+0.00000000e+00j,
       1.05847838e-08+0.00000000e+00j, 1.05847839e-08+0.00000000e+00j,
       8.12469946e-02+0.00000000e+00j, 8.12469946e-02+0.00000000e+00j,
       8.12469946e-02+0.00000000e+00j, 8.12469946e-02+0.00000000e+00j,
       8.12469946e-02+0.00000000e+00j, 8.12469946e-02+0.00000000e+00j,
       8.26670825e-02+0.00000000e+00j, 8.26670825e-02+0.00000000e+00j,
       8.26670825e-02+0.00000000e+00j, 8.26670825e-02+0.00000000e+00j,
       8.26670825e-02+0.00000000e+00j, 8.26670825e-02+0.00000000e+00j,
       3.36085923e-01+0.00000000e+00j, 3.36085923e-01+0.00000000e+00j,
       3.36085923e-01+0.00000000e+00j, 3.36085923e-01+0.00000000e+00j,
       3.36085923e-01+0.00000000e+00j, 3.36085923e-01+0.00000000e+00j,
       7.48001237e-01+0.00000000e+00j, 7.48001237e-01+0.00000000e+00j,
       7.51998753e-01+0.00000000e+00j, 7.51998753e-01+0.00000000e+00j])